In [35]:
!pip install resampy

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
!pip install np_utils

In [40]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

In [41]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from tqdm import tqdm
import resampy

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense , Activation , Dropout , SpatialDropout2D, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, ReLU, Flatten
from keras.optimizers import Adam

from keras import backend as keras_backend
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold

import seaborn as sns
import random
import math

from glob import glob
from itertools import cycle

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [42]:
def create_dataset_df(csv_file):
    dataset_df = pd.read_csv(csv_file)
    filepaths = []
    for i, row in dataset_df.iterrows():
        filepaths.append(os.path.join('/content/drive/MyDrive/ThesisData/', 'File'+str(row['fold']), row['slice_file_name']))
    dataset_df['filepath'] = filepaths
    return dataset_df

In [43]:
df = create_dataset_df('/content/drive/MyDrive/ThesisData/ThesisLionMD.csv')

In [44]:
df.head()

,fs_id,slice_file_name,classID,fold,class,filepath
0,iroar_123,iroar_123.wav,1,1,IRoar,/content/drive/MyDrive/ThesisData/File1/iroar_...
1,iroar_308,iroar_308.wav,1,1,IRoar,/content/drive/MyDrive/ThesisData/File1/iroar_...
2,grunt_878,grunt_878.wav,2,2,Grunt,/content/drive/MyDrive/ThesisData/File2/grunt_...
3,roar_178,roar_178.wav,0,1,Roar,/content/drive/MyDrive/ThesisData/File1/roar_1...
4,roar_963,roar_963.wav,0,3,Roar,/content/drive/MyDrive/ThesisData/File3/roar_9...


In [45]:
def get_duration(file_name):

    # load the audio file
    audio,sample_rate = librosa.load(file_name,res_type='kaiser_fast')

    time = librosa.get_duration(y=audio, sr=sample_rate)

    return time

In [46]:
duration = []

for index_num, row in tqdm(df.iterrows()):

    # get the file
    file_name = os.path.join(os.path.abspath('drive/MyDrive/ThesisData/'),'File'+str(row["fold"])+'/',str(row['slice_file_name']))

    time = get_duration(file_name)

    duration.append(time)

3480it [01:14, 46.92it/s]


In [47]:
df['duration'] = duration
sorted_df = df[df.duration <= 3]
sorted_df['class'].value_counts()

Grunt    1491
Roar      993
IRoar     987
Name: class, dtype: int64

In [48]:
len(sorted_df)

3471

In [49]:
columns = ['filepath', 'class']

In [50]:
filtered_df = sorted_df[columns]

In [52]:
filtered_df.head()

,filepath,class
0,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
1,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
2,/content/drive/MyDrive/ThesisData/File2/grunt_...,Grunt
3,/content/drive/MyDrive/ThesisData/File1/roar_1...,Roar
4,/content/drive/MyDrive/ThesisData/File3/roar_9...,Roar


In [53]:
df_sound = filtered_df[filtered_df["class"] == "IRoar"]

In [54]:
df_sound.head()

,filepath,class
0,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
1,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
8,/content/drive/MyDrive/ThesisData/File3/iroar_...,IRoar
10,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
11,/content/drive/MyDrive/ThesisData/File3/iroar_...,IRoar


In [56]:
len(df_sound)

987

In [58]:
# Set the maximum audio duration (in seconds)
max_audio_duration = 3

# Set the segment length (in seconds)
segment_length = 0.181406

# Initialize the desired number of segments
desired_num_segments = int(max_audio_duration / segment_length)

# Create an empty DataFrame to store the results
df = pd.DataFrame(columns=['File', 'Segment_Number', 'Start_Time', 'End_Time', 'Energy'])

# Parameters for segmentation
sampling_rate = 22050  # Sampling rate in Hz

In [59]:
audio_files = np.array(df_sound['filepath'])

In [60]:
# Iterate through the audio files
for audio_file in audio_files:
    # Load the audio file using librosa
    audio, sr = librosa.load(audio_file)

    # Calculate the number of samples for the segment length
    segment_samples = int(segment_length * sr)

    # Pad the audio to the maximum duration of 3 seconds
    max_samples = int(max_audio_duration * sr)
    audio = np.pad(audio, (0, max_samples - len(audio)), mode='constant')

    # Calculate the hop length based on the segment length
    constant_hop_length = segment_samples

    # Calculate the energy of each segment
    energy = np.array([sum(abs(audio[i:i+constant_hop_length])) for i in range(0, len(audio), constant_hop_length)])

    # Convert the energy values to numeric dtype
    energy = pd.to_numeric(energy)

    # Calculate the start and end times of each segment
    num_segments = len(energy)
    segment_duration = len(audio) / num_segments / sr  # Actual duration of each segment

    # All segments have the same start and end times
    start_times = np.arange(num_segments) * segment_duration
    end_times = start_times + segment_duration

    # Create a DataFrame for the current audio file
    audio_df = pd.DataFrame({'File': [audio_file] * num_segments,
                             'Segment_Number': np.arange(num_segments),
                             'Start_Time': start_times,
                             'End_Time': end_times,
                             'Energy': energy})

    # Append the results to the main DataFrame
    df = pd.concat([df, audio_df], ignore_index=True)

In [65]:
# Group by audio file and find the top 3 segments with maximum energy
top_3_segments = df.groupby('File').apply(lambda x: x.nlargest(3, 'Energy')).reset_index(drop=True)

# Print the top 3 segments for each audio file
print("Top 5 Segments for Each Audio File with Maximum Energy:")
print(top_3_segments)

Top 5 Segments for Each Audio File with Maximum Energy:
                                                   File Segment_Number  \
0     /content/drive/MyDrive/ThesisData/File1/iroar_...              5   
1     /content/drive/MyDrive/ThesisData/File1/iroar_...              6   
2     /content/drive/MyDrive/ThesisData/File1/iroar_...              3   
3     /content/drive/MyDrive/ThesisData/File1/iroar_...              8   
4     /content/drive/MyDrive/ThesisData/File1/iroar_...              7   
...                                                 ...            ...   
2956  /content/drive/MyDrive/ThesisData/File3/iroar_...              5   
2957  /content/drive/MyDrive/ThesisData/File3/iroar_...              8   
2958  /content/drive/MyDrive/ThesisData/File3/iroar_...              6   
2959  /content/drive/MyDrive/ThesisData/File3/iroar_...              5   
2960  /content/drive/MyDrive/ThesisData/File3/iroar_...              7   

      Start_Time  End_Time       Energy  
0       0.882

In [62]:
top_segment = df['Segment_Number'].value_counts()
print(top_segment)

0     987
9     987
15    987
14    987
13    987
12    987
11    987
10    987
8     987
1     987
7     987
6     987
5     987
4     987
3     987
2     987
16    987
Name: Segment_Number, dtype: int64


In [66]:
top_3_segments.head(15)

,File,Segment_Number,Start_Time,End_Time,Energy
0,/content/drive/MyDrive/ThesisData/File1/iroar_...,5,0.882353,1.058824,3436.693073
1,/content/drive/MyDrive/ThesisData/File1/iroar_...,6,1.058824,1.235294,3413.422944
2,/content/drive/MyDrive/ThesisData/File1/iroar_...,3,0.529412,0.705882,2447.790735
3,/content/drive/MyDrive/ThesisData/File1/iroar_...,8,1.411765,1.588235,3705.168778
4,/content/drive/MyDrive/ThesisData/File1/iroar_...,7,1.235294,1.411765,3656.852493
5,/content/drive/MyDrive/ThesisData/File1/iroar_...,6,1.058824,1.235294,3517.297911
6,/content/drive/MyDrive/ThesisData/File1/iroar_...,6,1.058824,1.235294,3397.811807
7,/content/drive/MyDrive/ThesisData/File1/iroar_...,7,1.235294,1.411765,2985.026644
8,/content/drive/MyDrive/ThesisData/File1/iroar_...,4,0.705882,0.882353,2311.198938
9,/content/drive/MyDrive/ThesisData/File1/iroar_...,7,1.235294,1.411765,3338.296505


In [68]:
# Using series value_counts()
top_segment = top_3_segments['Segment_Number'].value_counts()
print(top_segment)

5     623
6     535
4     492
7     380
3     315
8     201
2     114
9      87
1      62
10     46
11     36
0      25
12     24
13     14
14      6
15      1
Name: Segment_Number, dtype: int64


In [69]:
point_start = 0.882353*22050
point_end = 1.058824*22050
print(point_start, point_end)

19455.88365 23347.069199999998
